# Demo of ATS1 benchmark using Jupyter Notebook

This is a demo of FalCAuN on Jupyter Notebook using the [kotlin-jupyter kernel](https://github.com/Kotlin/kotlin-jupyter). This demo assumes that `jupyter` is executed with the following environmental variables.

- `JAVA_HOME` (the java home for Java 11)
- `KOTLIN_JUPYTER_JAVA_OPTS="-Djava.library.path=$MATLAB_HOME/bin/maca64/:$MATLAB_HOME/bin/maci64:$MATLAB_HOME/bin/glnxa64"`

In [1]:
KotlinVersion.CURRENT

2.0.0

## Dependent packages

This notebook depends on FalCAuN-core and FalCAuN-matlab

In [2]:
@file:DependsOn("net.maswag.falcaun:FalCAuN-core:1.0-SNAPSHOT")
@file:DependsOn("net.maswag.falcaun:FalCAuN-matlab:1.0-SNAPSHOT")

## Automatic transmission model

The following shows the common configuration to run the Automatic Transmission benchmark [Hoxha et al., ARCH@CPSWeek 2014].

* [Hoxha et al., ARCH@CPSWeek 2014]: *Benchmarks for Temporal Logic Requirements for Automotive Systems*, ARCH@CPSWeek 2014, Bardh Hoxha, Houssam Abbas, Georgios E. Fainekos

In [3]:
import net.maswag.falcaun.*

val initScript = """
versionString = version('-release');
oldpath = path;
path(strcat(userpath, '/Examples/R', versionString, '/simulink_automotive/ModelingAnAutomaticTransmissionControllerExample/'), oldpath);

mdl = 'Autotrans_shift';
load_system(mdl);
"""
val paramNames = listOf("throttle", "brake")
val signalStep = 1.0
val simulinkSimulationStep = 0.0025

In [4]:
// Load the automatic transmission model. This must be manually closed!!
val sul = SimulinkSUL(initScript, paramNames, signalStep, simulinkSimulationStep)

## Definition of the STL properties

In [5]:
import java.io.BufferedReader
import java.io.StringReader

// Define the input and output mappers
val throttleValues = listOf(0.0, 100.0)
val brakeValues = listOf(0.0, 325.0)
val inputMapper = InputMapperReader.make(listOf(throttleValues, brakeValues))
val ignoreValues = listOf(null)
val velocityValues = listOf(20.0, 40.0, 60.0, 80.0, 100.0, 120.0, null)
val accelerationValues = listOf(null)
val gearValues = listOf(null)
val outputMapperReader = OutputMapperReader(listOf(ignoreValues, accelerationValues, gearValues, velocityValues))
outputMapperReader.parse()
val mapperString = listOf("previous_max_output(0)").joinToString("\n")
val signalMapper: ExtendedSignalMapper = ExtendedSignalMapper.parse(BufferedReader(StringReader(mapperString)))
assert(signalMapper.size() == 1)
val mapper =
    NumericSULMapper(inputMapper, outputMapperReader.largest, outputMapperReader.outputMapper, signalMapper)

In [6]:
import net.maswag.falcaun.TemporalLogic.STLCost;

// Define the STL properties
val stlFactory = STLFactory()
val stlList: List<STLCost> = listOf(
    "[] (signal(3) < 120)",
    "[] (signal(3) < 100)",
    "[] (signal(3) < 80)",
    "[] (signal(3) < 60)",
    "[] (signal(3) < 40)",
    "[] (signal(3) < 20)"
).map { stlString ->
    stlFactory.parse(
        stlString,
        inputMapper,
        outputMapperReader.outputMapper,
        outputMapperReader.largest
    )
}.toList()
val signalLength = 30
val properties = AdaptiveSTLList(stlList, signalLength)

## Configure the verifier

In [7]:
val verifier = NumericSULVerifier(sul, signalStep, properties, mapper)

// Timeout must be set before adding equivalence testing
verifier.setTimeout(5 * 60) // 5 minutes

### Configure the equivalence testing

In this demo, we use the equivalence testing based on an genetic algorithm. The following defines the constants.

In [8]:
// Constants for the GA-based equivalence testing
val maxTest = 50000
val populationSize = 200
val crossoverProb = 0.5
val mutationProb = 0.01

verifier.addGAEQOracleAll(
    signalLength,
    maxTest,
    ArgParser.GASelectionKind.Tournament,
    populationSize,
    crossoverProb,
    mutationProb
)

## Run the verifier

Then, we run the verifier. This takes some minutes.

In [9]:
val result = verifier.run()
result

false

In [10]:
import net.automatalib.word.Word;

var rawOutput = mutableListOf<List<List<Double>>>()
for (i in 0 until verifier.cexProperty.size) {
    val dim = mutableListOf<List<Double>>()
    for (j in 0 until verifier.cexConcreteInput[i].size()) {
        dim.add(verifier.cexConcreteInput[i].get(j))
    }
    val inputWord = Word.fromList(dim)
    val resultWord = sul.execute(inputWord).getOutputSignal()
    rawOutput.add(resultWord.asList())
}

## Print the result

The following prints some information about the falsified properties. Notice that 5 minutes is typically too short to falsify all the properties in ATS1, and some of them are still unfalsified.

In [11]:
// Print the result
for (i in 0 until verifier.cexProperty.size) {
    println("${verifier.cexProperty[i]} is falsified by the following counterexample)")
    println("cex concrete input: ${verifier.cexConcreteInput[i]}")
    println("cex abstract input: ${verifier.cexAbstractInput[i]}")
    println("cex concrete output: ${rawOutput[i]}")
    println("cex abstract output: ${verifier.cexOutput[i]}")
}

[] ( output(3) < 100.000000 ) is falsified by the following counterexample)
cex concrete input: [0.0 100.0 0.0; 1.0 100.0 0.0; 2.0 100.0 0.0; 3.0 100.0 0.0; 4.0 100.0 0.0; 5.0 100.0 0.0; 6.0 100.0 325.0; 7.0 100.0 0.0; 8.0 100.0 0.0; 9.0 100.0 0.0; 10.0 100.0 0.0; 11.0 100.0 0.0; 12.0 100.0 0.0; 13.0 100.0 0.0]
cex abstract input: ba ba ba ba ba ba bb ba ba ba ba ba ba ba
cex concrete output: [[0.0, 1000.0, 1.0], [20.422757349670167, 2854.42992716515, 1.0], [34.33594388511233, 3987.5492700067966, 1.0], [46.24543927046309, 3410.4909936983454, 2.0], [54.56557128409135, 3880.3643902078347, 2.0], [62.37843248393348, 4332.148130421007, 2.0], [69.5742659461339, 4713.76777766446, 2.0], [73.93607346276868, 3688.9888472216894, 3.0], [79.16190202894639, 3887.073610910538, 3.0], [84.15270347104955, 4089.0299316433993, 3.0], [88.92254422702149, 4278.300475797851, 3.0], [93.47438630393462, 4453.17618059262, 3.0], [97.73584457511679, 4608.0485570032715, 3.0], [101.80239672949082, 3494.210469514378, 

## Plot the result

We can also plot the result of the falsification using `lets-plot`. We remark that we use the discrete-time semantics, and the results shown here are also rather coarsely sampled (by default, 1.0-time unit).

In [12]:
%use lets-plot

In [13]:
val bunch = GGBunch()
    for (i in 0 until verifier.cexProperty.size) {
        val size = verifier.cexConcreteInput[i].size()
        val datasetIn = mapOf(
            "time" to List(size) { it.toDouble() } + List(size) { it.toDouble() },
            "input" to verifier.cexConcreteInput[i].dimensionGet(0) + verifier.cexConcreteInput[i].dimensionGet(1),
            "group" to List(size) {"throttle"} + List(size) {"brake"}
        )
        bunch.addPlot(letsPlot(datasetIn) + 
                geomPath(showLegend = true) {x = "time"; y = "input"; color = "group"} +
                labs(title = "Input to falsify" + verifier.cexProperty[i]), 0, 600 * i, 1000, 200)
        val datasetOut = mapOf(
            "time" to (0 until size).flatMap { value -> List(3) { value.toDouble() } },
            "output" to rawOutput[i].flatten(),
            "group" to (1..size + 1).flatMap { listOf("velocity", "rotation", "gear") }.take(size * 3)
        )
        bunch.addPlot(letsPlot(datasetOut) + 
                geomPath(showLegend = true) {x = "time"; y = "output"; color = "group"} +
                labs(title = "Output to falsify" + verifier.cexProperty[i]), 0, 600 * i + 200, 1000, 200)
        val indicesToExclude = datasetOut["group"]!!.withIndex().filter { it.value == "rotation" }.map { it.index }
        val noRotation = datasetOut.mapValues { (key, value) ->
            if (value is List<*>) value.withIndex().filterNot { it.index in indicesToExclude }.map { it.value }
            else value
        }
        bunch.addPlot(letsPlot(noRotation) + 
                geomPath(showLegend = true) {x = "time"; y = "output"; color = "group"} +
                labs(title = "Output to falsify" + verifier.cexProperty[i] + " (without rotation for visualization)"), 0, 600 * i + 400, 1000, 200)
    }
bunch.show()

## IMPORTANT!! Close MATLAB engine

The following terminates the MATLAB engine. This must be executed at the end. Otherwise, the MATLAB process remains running. If you want to run the Simulink model, you can just re-initialize it.

In [14]:
sul.close()